<a href="https://colab.research.google.com/github/as2c/MLP---Cosmology/blob/main/Data2/density_contrast_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
!git clone https://github.com/jveitchmichaelis/pygadgetreader

Cloning into 'pygadgetreader'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 236 (delta 6), reused 236 (delta 6), pack-reused 0
Receiving objects: 100% (236/236), 749.63 KiB | 28.83 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/pygadgetreader
!python setup.py build
!python setup.py install

/content/pygadgetreader
running build
running build_py
running install
running bdist_egg
running egg_info
writing pyGadgetReader.egg-info/PKG-INFO
writing dependency_links to pyGadgetReader.egg-info/dependency_links.txt
writing requirements to pyGadgetReader.egg-info/requires.txt
writing top-level names to pyGadgetReader.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'pyGadgetReader.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/readgadget
copying build/lib/readgadget/readrockstar.py -> build/bdist.linux-x86_64/egg/readgadget
creating build/bdist.linux-x86_64/egg/readgadget/modules
copying build/lib/readgadget/modules/common.py -> build/bdist.linux-x86_64/egg/readgadget/modules
copying build/lib/readgadget/modules/hdf5.py -> build/bdist.linux-x86_64/egg/readgadget/modules
copying bui

In [5]:
from pygadgetreader import *

In [6]:
# vel = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','vel','dm')
# vx = vel[:,0].tolist()
# vy = vel[:,1].tolist()
# vz = vel[:,2].tolist()
# mass = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','mass','dm').tolist()
# particle_id = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','pid','dm').tolist()
# position = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','pos','dm')
# x = position[:,0].tolist()
# y = position[:,1].tolist()
# z = position[:,2].tolist()
# data = {'particle_id': particle_id,
#         'x': x,
#         'y': y,
#         'z': z,
#         'vx': vx,
#         'vy': vy,
#         'vz': vz,
#         'mass': mass}
# df = pd.DataFrame(data)from pygadgetreader import *

# seed = int(input("Enter the seed:"))
halopath = '/content/drive/MyDrive/ML/Project/halos_0'
initial_snap_path = '/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm'

#Reading the halo particles 

particles = readrockstar(halopath,'particles')
halo_particles = pd.DataFrame()
halo_particles['id'] = particles[:,0]
halo_particles['haloID'] = particles[:,1]

#Reading the halo masses
halo_num = pd.DataFrame() 
halos = readrockstar(halopath, 'halos') 
halo_num['haloID'] = halos['id']
halo_num['np'] = halos['num_p']

#a dataframe containing the subhalo id and sub halo mass for each particle
halo_particles = pd.merge(halo_particles, halo_num , right_on = 'haloID', left_on = 'haloID', how = 'outer')

print(halo_particles.describe())

                 id        haloID            np
count  2.214619e+06  2.214619e+06  2.214619e+06
mean   8.946561e+06  6.562720e+02  4.008056e+03
std    4.967033e+06  3.655899e+02  5.164701e+03
min    1.000000e+01  0.000000e+00  2.020000e+02
25%    4.549009e+06  3.370000e+02  1.218000e+03
50%    9.372965e+06  6.610000e+02  2.190000e+03
75%    1.337186e+07  9.760000e+02  4.205000e+03
max    1.677717e+07  1.272000e+03  3.125200e+04


In [7]:
particle_data = pd.DataFrame()

particle_data['id'] = np.array(readsnap(initial_snap_path, 'pid', 'dm'))

particle_data['x'] = np.array(readsnap(initial_snap_path, 'pos', 'dm'))[:,0]
particle_data['y'] = np.array(readsnap(initial_snap_path, 'pos', 'dm'))[:,1]
particle_data['z'] = np.array(readsnap(initial_snap_path, 'pos', 'dm'))[:,2]

particle_data['vx'] = np.array(readsnap(initial_snap_path, 'vel', 'dm'))[:,0]
particle_data['vy'] = np.array(readsnap(initial_snap_path, 'vel', 'dm'))[:,1]
particle_data['vz'] = np.array(readsnap(initial_snap_path, 'vel', 'dm'))[:,2]

Returning DM    Particle IDs
Returning DM    Positions
Returning DM    Positions
Returning DM    Positions
Returning DM    Velocities
Returning DM    Velocities
Returning DM    Velocities


In [8]:
data = pd.merge(particle_data, halo_particles, left_on = 'id', right_on = 'id', how = 'outer')
data['haloID'] = data['haloID'].fillna(0)
data['halo_mass'] = data['np'].fillna(0)
data = data[['id','x','y','z','vx','vy','vz','haloID','halo_mass']]

In [9]:
data

,id,x,y,z,vx,vy,vz,haloID,halo_mass
0,0,192.691193,231.858017,225.984268,-659.554321,-419.040802,-489.896637,0.0,0.0
1,7,809.101929,210.431351,214.217163,-702.608582,-530.673584,-534.257935,0.0,0.0
2,56,1437.270874,198.266098,201.348267,-689.156860,-608.542847,-615.787476,0.0,0.0
3,63,2043.859375,196.952347,187.470413,-797.107727,-646.467102,-686.565735,0.0,0.0
4,64,2664.605225,204.039200,170.532455,-818.485474,-626.312866,-768.481689,0.0,0.0
...,...,...,...,...,...,...,...,...,...
16777211,16777133,157062.375000,159616.125000,159625.109375,-701.834167,-397.481934,-357.832153,0.0,0.0
16777212,16777170,157688.234375,159621.156250,159610.312500,-678.720825,-381.776428,-408.717957,0.0,0.0
16777213,16777173,158317.078125,159623.343750,159602.531250,-681.724609,-368.299011,-465.415466,0.0,0.0
16777214,16777194,158937.750000,159618.203125,159612.140625,-697.889648,-382.313843,-450.085297,0.0,0.0


In [10]:
pd.set_option('display.float_format', '{:e}'.format)

In [11]:
#converting the postion from kpc to Mpc
data['x'] = data['x']/1000
data['y'] = data['y']/1000
data['z'] = data['z']/1000
data = data.astype({'haloID':'int64'})
data = data.sort_values(by = 'x')

In [12]:
data

,id,x,y,z,vx,vy,vz,haloID,halo_mass
2573056,3702803,1.043876e-01,2.210869e+01,3.653300e+01,-1.147800e+03,-4.212986e+02,-1.593734e+02,0,0.000000e+00
2630144,4227097,1.063551e-01,2.149263e+01,4.091941e+01,-1.127608e+03,-4.015528e+02,-1.114685e+02,0,0.000000e+00
11749888,10133520,1.074259e-01,1.065825e+02,9.652391e+01,-1.146398e+03,1.358670e+02,-2.276244e+02,0,0.000000e+00
11750144,10133523,1.084110e-01,1.071977e+02,9.652399e+01,-1.141314e+03,4.801611e+01,-2.313971e+02,0,0.000000e+00
2654464,4489219,1.084628e-01,2.084463e+01,4.277568e+01,-1.133097e+03,-5.158618e+02,-2.124474e+02,0,0.000000e+00
...,...,...,...,...,...,...,...,...,...
10339839,15605748,1.598403e+02,8.340266e+01,1.489850e+02,8.447092e+02,-1.752089e+02,-4.431822e+02,915,5.946000e+03
10348031,15605749,1.598430e+02,8.341740e+01,1.496117e+02,8.598524e+02,-9.485448e+01,-4.641362e+02,915,5.946000e+03
10340351,15605740,1.598443e+02,8.463966e+01,1.490094e+02,8.691870e+02,-2.896236e+02,-3.002645e+02,915,5.946000e+03
10348287,15605742,1.598458e+02,8.405312e+01,1.496214e+02,8.852476e+02,-6.958143e+01,-3.914064e+02,916,7.980000e+03


In [13]:
box_size = 159
boundary = 20

d = data[data.x > boundary]

d = d[d.x < box_size - boundary]

d = d[d.y > boundary]

d = d[d.y < box_size - boundary]

d = d[d.z > boundary]

d = d[d.z < box_size - boundary]

In [14]:
d.shape

(6873748, 9)

In [15]:
from tqdm import tqdm
def box_mass(r_cutoff, data, sample):
    m = len(sample)
    mass = pd.DataFrame(np.zeros(m),columns=['den_contrast'])
    for i in tqdm(range(m)):
        wx=sample.iloc[i]['x']
        wy=sample.iloc[i]['y']
        wz=sample.iloc[i]['z']
        ix = np.searchsorted(data[:,1] , wx-r_cutoff)
        jx = np.searchsorted(data[:,1] , wx+r_cutoff)

        temp = np.where((data[ix:jx,2] < wy+r_cutoff) & (data[ix:jx,2] > wy-r_cutoff) & (data[ix:jx,3] < wz+r_cutoff) & (data[ix:jx,3] > wz-r_cutoff))
    
        mass.den_contrast[i] = len(temp[0])
    return mass

In [16]:
d.shape

(6873748, 9)

In [17]:
n = 100

sample = d.sample(n, random_state = 42)
sample_density = sample[['id', 'haloID']]
sample_density.reset_index(drop = True, inplace = True)

In [18]:
sample_density.head()

,id,haloID
0,12445545,460
1,9757813,0
2,4364083,0
3,9111430,0
4,2698019,0


In [19]:
print(data)

                id            x            y  ...            vz  haloID    halo_mass
2573056    3702803 1.043876e-01 2.210869e+01  ... -1.593734e+02       0 0.000000e+00
2630144    4227097 1.063551e-01 2.149263e+01  ... -1.114685e+02       0 0.000000e+00
11749888  10133520 1.074259e-01 1.065825e+02  ... -2.276244e+02       0 0.000000e+00
11750144  10133523 1.084110e-01 1.071977e+02  ... -2.313971e+02       0 0.000000e+00
2654464    4489219 1.084628e-01 2.084463e+01  ... -2.124474e+02       0 0.000000e+00
...            ...          ...          ...  ...           ...     ...          ...
10339839  15605748 1.598403e+02 8.340266e+01  ... -4.431822e+02     915 5.946000e+03
10348031  15605749 1.598430e+02 8.341740e+01  ... -4.641362e+02     915 5.946000e+03
10340351  15605740 1.598443e+02 8.463966e+01  ... -3.002645e+02     915 5.946000e+03
10348287  15605742 1.598458e+02 8.405312e+01  ... -3.914064e+02     916 7.980000e+03
10340095  15605743 1.598468e+02 8.405097e+01  ... -3.831090e+02  

In [20]:
data = data.to_numpy()

In [21]:
data = data.astype('float64')

In [22]:
data.shape

(16777216, 9)

In [23]:
n = 100000

sample = d.sample(n, random_state = 42)
sample_density = sample[['id', 'haloID']]
sample_density.reset_index(drop = True, inplace = True)

for r in np.linspace(4.9, 20, 9):
  density = box_mass(r , data, sample)
  density = density.div((2*r)**3)
  sample_density = pd.concat([sample_density, density], axis = 1)
  sample_density = sample_density.rename(columns={'den_contrast': 'dc for {0:.1f}'.format(r)})
sample_density.head()

100%|██████████| 100000/100000 [43:14<00:00, 38.54it/s]


,id,haloID,dc for 4.9,dc for 6.8,dc for 8.7,dc for 10.6,dc for 12.4,dc for 14.3,dc for 16.2,dc for 18.1,dc for 20.0
0,12445545,460,3.747376e+00,3.940263e+00,4.009581e+00,4.097859e+00,4.118019e+00,4.116688e+00,4.123362e+00,4.141595e+00,4.153437e+00
1,9757813,0,3.607128e+00,3.787961e+00,3.893933e+00,3.988391e+00,4.016777e+00,4.020242e+00,4.025908e+00,4.035507e+00,4.045141e+00
2,4364083,0,3.680439e+00,3.839128e+00,3.935673e+00,3.947870e+00,3.951873e+00,3.968627e+00,3.992516e+00,4.038915e+00,4.063891e+00
3,9111430,0,3.594378e+00,3.870308e+00,3.919207e+00,3.965903e+00,4.025327e+00,4.084115e+00,4.125586e+00,4.124197e+00,4.119031e+00
4,2698019,0,3.591191e+00,3.715608e+00,3.777136e+00,3.839462e+00,3.902256e+00,3.975795e+00,4.031615e+00,4.063465e+00,4.080422e+00


In [24]:
sample_density.describe()

,id,haloID,dc for 4.9,dc for 6.8,dc for 8.7,dc for 10.6,dc for 12.4,dc for 14.3,dc for 16.2,dc for 18.1,dc for 20.0
count,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05
mean,8.336331e+06,7.351837e+01,3.608464e+00,3.755829e+00,3.853160e+00,3.922281e+00,3.974161e+00,4.014609e+00,4.047026e+00,4.073129e+00,4.093753e+00
std,3.601997e+06,2.368330e+02,4.754085e-02,6.682156e-02,7.406519e-02,7.351311e-02,6.926706e-02,6.338634e-02,5.725504e-02,5.147849e-02,4.617630e-02
min,2.130504e+06,0.000000e+00,3.585878e+00,3.702017e+00,3.768711e+00,3.811989e+00,3.842340e+00,3.864802e+00,3.882449e+00,3.901799e+00,3.924484e+00
25%,5.190622e+06,0.000000e+00,3.585878e+00,3.705215e+00,3.791496e+00,3.862984e+00,3.921818e+00,3.969009e+00,4.008056e+00,4.039820e+00,4.064109e+00
50%,8.323708e+06,0.000000e+00,3.585878e+00,3.728000e+00,3.835534e+00,3.913714e+00,3.970398e+00,4.014050e+00,4.047038e+00,4.073405e+00,4.094391e+00
75%,1.143864e+07,0.000000e+00,3.606065e+00,3.784364e+00,3.897762e+00,3.969403e+00,4.020339e+00,4.056929e+00,4.085412e+00,4.107053e+00,4.124422e+00
max,1.464666e+07,1.233000e+03,4.086308e+00,4.238072e+00,4.247387e+00,4.260471e+00,4.274578e+00,4.254484e+00,4.242765e+00,4.251363e+00,4.250687e+00


In [25]:
sample_density.to_csv('/content/drive/MyDrive/sample_density_final2.csv', index=False)

In [26]:
rho_bar = len(data) / (159.8**3)
sample_density.iloc[:,2:] -= rho_bar
sample_density.iloc[:,2:] /= rho_bar
print(sample_density)

             id  haloID    dc for 4.9  ...   dc for 16.2   dc for 18.1   dc for 20.0
0      12445545     460 -8.853990e-02  ...  2.909874e-03  7.344509e-03  1.022499e-02
1       9757813       0 -1.226518e-01  ... -2.079352e-02 -1.845873e-02 -1.611565e-02
2       4364083       0 -1.048206e-01  ... -2.891532e-02 -1.762983e-02 -1.155516e-02
3       9111430       0 -1.257529e-01  ...  3.450852e-03  3.113044e-03  1.856490e-03
4       2698019       0 -1.265282e-01  ... -1.940548e-02 -1.165871e-02 -7.534323e-03
...         ...     ...           ...  ...           ...           ...           ...
99995   5727559       0 -1.231687e-01  ... -2.297879e-02 -1.671395e-02 -1.403682e-02
99996  13779938       0 -1.278203e-01  ...  2.667857e-03  5.973249e-03  6.515793e-03
99997   5313693       0 -1.278203e-01  ... -1.696396e-02 -6.414148e-03 -2.183346e-03
99998   7482364       0 -1.278203e-01  ... -1.503495e-02 -1.427843e-02 -1.217462e-02
99999   8333714       0 -1.278203e-01  ... -5.169211e-03 -2.33241

In [27]:
sample_density.to_csv('/content/drive/MyDrive/density_contrast_final2.csv', index=False)

The above gives the physical properties that we were looking for, namely the density contrasts for each particle encompassed by boxes of different sizes.